In [1]:
!pip install -q lxml

In [11]:
import bs4
import lxml
import pandas
import urllib
from tqdm.auto import tqdm

from urllib import request

In [51]:
""" Scrapping du site insee.fr """

url = 'https://www.insee.fr/fr/statistiques/3677771?sommaire=3677855'
raw_text = request.urlopen(url).read()
page = bs4.BeautifulSoup(raw_text) #,'lxml')
tableau = page.find('table', {'id' : 'produit-tableau-figure1'})
rows = tableau.find_all('tr')

In [52]:
""" Création d'un dictionnaire pour manipuler plus aisément les données """

data_pop = {
    rows[i].text.split("\n")[1] : [rows[i].text.split("\n")[2], rows[1].text.split("\n")[3].replace('\xa0',' ')] for i in range(1,len(rows))
}

In [19]:
""" Conversion en DataFrame pour exporter """

df_pop = pandas.DataFrame(data_pop)

In [20]:
""" Exportation au format json """

df_pop.to_json('data_pop.json')

In [228]:
""" Définition de la classe finale utilisée """

class Population_departement :
    """
    Class permettant de scrapper et d'effectuer des opérations sur les données de l'INSEE concernant la population officielle des départements entre
    2011 et 2020.
    
    """
    def __init__ (self) :
        """
        Initialisation de la classe contenant :
            self.data_pop : Dictionnaire contenant les données
            self.urls : liste des urls nécessaire au scrapping
        """
        self.data_pop = {
            i : {} for i in range(2011,2021)
            }
        self.urls = [
            'https://www.insee.fr/fr/statistiques/2119745?sommaire=2119751',
            'https://www.insee.fr/fr/statistiques/2119678?sommaire=2119686',
            'https://www.insee.fr/fr/statistiques/2119468?sommaire=2119504',
            'https://www.insee.fr/fr/statistiques/2525762?sommaire=2525768',
            'https://www.insee.fr/fr/statistiques/3292643?sommaire=3292701',
            'https://www.insee.fr/fr/statistiques/3677771?sommaire=3677855',#
            'https://www.insee.fr/fr/statistiques/4265390?sommaire=4265511',
            'https://www.insee.fr/fr/statistiques/4989753?sommaire=4989761',
            'https://www.insee.fr/fr/statistiques/6013867?sommaire=6011075',
            'https://www.insee.fr/fr/statistiques/6683015?sommaire=6683037'
        ]

    def scrap(self):
        """ 
        Fonction permettant de scrapper les données voulues, i.e. la population officielle de chaque département de 2011 à 2020 depuis le site
        de l'INSEE.

            Paramètres:
            ------------
                self.data_pop : dict
                    Dictionnaire contenant toutes les données par années
        
            Output:
            ----------
                self.data_pop : dict
                    Dictionnaire contenant toutes les données par années actualisées
            
        """
        for i in range (len(self.urls)) :
            url = self.urls[i]
            raw_text = request.urlopen(url).read()
            page = bs4.BeautifulSoup(raw_text) #,'lxml')
            if i < 6 :
                fig = '1'
            else :
                fig = '2'
            tableau = page.find('table', {'id' : 'produit-tableau-figure' + fig})
            rows = tableau.find_all('tr')
            
            if 2011 + i < 2015 :
                self.data_pop[2011 + i] = {
                rows[j].text.split("\n")[1] : int(rows[j].text.split("\n")[3].replace('\xa0','').replace(' ','')) for j in range(3,len(rows))
                }
            elif 2011 + i == 2017 :
                self.data_pop[2011 + i] = {
                '0'+rows[j].text.split("\n")[1] : int(rows[j].text.split("\n")[3].replace('\xa0','').replace(' ','')) for j in range(1,10)
                }
                for j in range (10,len(rows)):
                    self.data_pop[2011 + i][rows[j].text.split("\n")[1]] = int(rows[j].text.split("\n")[3].replace('\xa0','').replace(' ',''))
            else :
                self.data_pop[2011 + i] = {
                    rows[j].text.split("\n")[1] : int(rows[j].text.split("\n")[3].replace('\xa0','').replace(' ','')) for j in range(1,len(rows))
                }
        

            
    def export(self) :
        """
        Fonction intégrée d'export pour permettant de faciliter les manipulations.

        Paramètres:
            ------------
                self.df_pop : DataFrame
                    Dictionnaire contenant toutes les données par années
        
            Output:
            ----------
                Aucun
        """
        self.df_pop = pandas.DataFrame(self.data_pop)
        self.df_pop.to_json('data_pop.json')